# Simulation and Noise Models

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '../')

We can set up a noise model, following the [Qiskit textbook](https://qiskit.org/textbook/ch-quantum-hardware/error-correction-repetition-code.html#Correcting-errors-in-qubits). We define our noise model to have equal chances of X and Z:

In [3]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

def get_noise(p_err):

    error_gate1 = pauli_error([('X', p_err/2), ('Z', p_err/2), ('I', 1 - p_err)])
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_gate1, "measure") # measurement error is applied to measurements
    noise_model.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"]) # single qubit gate error is applied to x gates
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"]) # two qubit gate error is applied to cx gates
        
    return noise_model

In [4]:
from qiskit import execute, Aer
from surface_code.circuits import SurfaceCode
from surface_code.fitters import GraphDecoder

We set up a test harness to run through some examples:

In [ ]:
def full_loop(code, decoder_simulated, decoder_analytic, noise_model=None):
    # noise_model = get_noise(0.01)
    counts = execute(code.circuit['0'], Aer.get_backend('qasm_simulator'), shots=1, noise_model=noise_model).result().get_counts()
    X_errors, Z_errors = code.extract_nodes(code.process_results(counts))

    print("Raw readout:", counts)
    print("Parsed X:", X_errors)
    print("Parsed Z:", Z_errors)
    
    # Analytic Decoder
    flips = {}
    for error_key,errors in dict(zip(("X", "Z"), (X_errors,Z_errors))).items():
        if errors:
            error_graph, paths = decoder_analytic.make_error_graph(errors, error_key)
            matching_graph = decoder_analytic.matching_graph(error_graph,error_key)
            matches = decoder_analytic.matching(matching_graph,error_key)
            flips[error_key] = decoder_analytic.calculate_qubit_flips(matches, paths,error_key)
        else:
            flips[error_key] = {}
    print("----------------")
    print("Decoder results (analytic):")
    print(decoder_analytic.net_qubit_flips(flips["X"], flips["Z"]))

    
    # Simulated Decoder 
    flips = {}
    for error_key,errors in dict(zip(("X", "Z"), (X_errors,Z_errors))).items():
        if errors:
            error_graph = decoder_simulated.make_error_graph(errors, error_key)#when using a simulated syndrome graph, make_error_graph will only return error_graph
            matching_graph = decoder_simulated.matching_graph(error_graph,error_key)
            matches = decoder_simulated.matching(matching_graph,error_key)
            paths = decoder_simulated.analytic_paths(matches,error_key) #simulated syndrome graph does not have nearest neighbor edges that allow for proper error chain determination
            flips[error_key] = decoder_simulated.calculate_qubit_flips(matches, paths,error_key)
        else:
            flips[error_key] = {}
    print("----------------")
    print("Decoder results (simulated):")
    print(decoder_simulated.net_qubit_flips(flips["X"], flips["Z"]))

In [ ]:
code = SurfaceCode(3, 3)
decoder_simulated = GraphDecoder(3, 3, simulation=True)
decoder_analytic = GraphDecoder(3, 3)

We can compare `decoder1`, the syndrome graph from simulation, with `decoder2`, the analytic graph:

In [ ]:
for i in range(10):
    print("====Run", i, "====")
    full_loop(code, decoder_simulated, decoder_analytic)
    print("=============", "\n")

And now run with some noise:

In [ ]:
for i in range(10):
    print("====Run", i, "====")
    full_loop(code, decoder_simulated, decoder_analytic, noise_model=get_noise(0.05))
    print("=============", "\n")

# Logical Error Rate Graphs

In [5]:
from surface_code.benchmarking import SurfaceCodeBenchmarkingTool

In [26]:
decoder = GraphDecoder(5,3) 
tool = SurfaceCodeBenchmarkingTool(decoder)
#'1 01010001 01010001 01010000'
results = {'0 10000000 10000000': 6197,
 '0 11100000 00100000': 2,
 '0 01010000 01010000': 6186,
 '0 00110100 00110000': 2,
 '0 01110011 01110000': 1,
 '0 01010010 01010000': 3,
 '0 11100111 11100000': 6,
 '0 00100110 00100000': 2,
 '0 11110010 11110000': 4,
 '1 00010010 00010000': 3,
 '0 01010001 01010000': 2,
 '1 10010010 10010000': 3,
 '0 00110111 00110000': 5,
 '0 00010111 00010000': 3,
 '0 10100000 11100000': 8,
 '0 01000111 01000000': 6,
 '0 01001000 01000000': 6,
 '0 10100111 10100000': 3,
 '0 11101010 11100000': 3,
 '0 01000000 00100000': 1,
 '0 10000111 10000000': 1,
 '0 01100000 01010000': 3,
 '0 00000001 00000000': 2,
 '0 00001001 00000000': 2,
 '0 11110100 11110000': 6,
 '0 11010110 11010000': 7,
 '0 11010000 11110000': 14,
 '0 01000000 00000000': 7,
 '0 01100000 11100000': 4,
 '0 00010011 00010000': 1,
 '0 11111010 11110000': 1,
 '0 11000010 11000000': 4,
 '0 11100000 01100000': 6,
 '0 01101010 01100000': 5,
 '0 01011001 01010000': 5,
 '0 00100000 11100000': 3,
 '0 10011001 10010000': 2,
 '0 01110001 01110000': 9,
 '0 01000100 01000000': 2,
 '0 11100000 11000000': 7,
 '0 10011000 10010000': 8,
 '0 01010000 01110000': 7,
 '0 10100000 01100000': 5,
 '0 01111010 01110000': 1,
 '0 10010100 10010000': 2,
 '0 00110000 00000000': 5,
 '0 11110111 11110000': 1,
 '0 01100100 01100000': 4,
 '0 11000000 10100000': 2,
 '0 00111010 00110000': 3,
 '0 01111001 01110000': 1,
 '0 10000010 10000000': 3,
 '0 00000000 00000000': 6249,
 '1 00010001 00010000': 4,
 '0 00110001 00110000': 6,
 '0 10100000 10110000': 5,
 '0 10110110 10110000': 6,
 '0 10111010 10110000': 3,
 '0 10110000 00110000': 7,
 '1 01000001 01000000': 7,
 '0 11100000 11100000': 6050,
 '0 10000000 11000000': 10,
 '0 00010001 00010000': 2,
 '0 10110000 10010000': 12,
 '0 10111000 10110000': 16,
 '0 01111000 01110000': 11,
 '0 11001000 11000000': 10,
 '0 11011010 11010000': 2,
 '0 01010000 01000000': 12,
 '0 11100000 10000000': 3,
 '0 00111001 00110000': 1,
 '1 00110001 00110000': 4,
 '0 01100000 00100000': 8,
 '0 00000000 01000000': 6,
 '0 10000100 10000000': 1,
 '0 10010000 11010000': 11,
 '0 10000000 00000000': 6,
 '0 00100100 00100000': 1,
 '0 10100000 00100000': 10,
 '0 11100001 11100000': 4,
 '0 10010000 10010000': 6122,
 '0 01000010 01000000': 1,
 '0 11110001 11110000': 2,
 '0 01001010 01000000': 6,
 '0 01011010 01010000': 1,
 '0 11000000 11000000': 6076,
 '0 11100000 10100000': 12,
 '0 00010000 10010000': 5,
 '0 00010000 01110000': 4,
 '0 00100000 00110000': 5,
 '0 10010000 00010000': 5,
 '0 01100000 10100000': 1,
 '0 01101000 01100000': 6,
 '0 00010000 00010000': 6294,
 '0 01110000 11110000': 4,
 '0 10110000 01110000': 6,
 '0 11110000 11000000': 2,
 '0 11100010 11100000': 6,
 '0 00000000 00100000': 8,
 '0 00000000 00110000': 2,
 '0 10010110 10010000': 2,
 '1 01110010 01110000': 2,
 '1 00100001 00100000': 6,
 '1 10010001 10010000': 14,
 '0 00111000 00110000': 8,
 '0 10110111 10110000': 6,
 '0 10100110 10100000': 3,
 '0 11110000 10110000': 12,
 '0 10110000 11010000': 6,
 '1 11100001 11100000': 8,
 '0 01100110 01100000': 2,
 '0 10010000 10000000': 7,
 '0 11110000 10010000': 2,
 '0 11110000 11110000': 6262,
 '0 00001010 00000000': 2,
 '0 10100000 10010000': 3,
 '0 00100000 10100000': 6,
 '0 00110000 00100000': 4,
 '1 01100010 01100000': 3,
 '0 01010000 11010000': 9,
 '1 11110010 11110000': 2,
 '0 01000000 01100000': 6,
 '0 01110000 10110000': 2,
 '0 10010000 01010000': 3,
 '0 00011010 00010000': 5,
 '0 11110000 11010000': 9,
 '0 00000000 11000000': 1,
 '0 11000100 11000000': 4,
 '1 10100010 10100000': 1,
 '0 10010111 10010000': 1,
 '0 00011001 00010000': 4,
 '0 00110000 00110000': 6182,
 '0 10000000 10010000': 8,
 '0 10110000 10110000': 6120,
 '0 01100001 01100000': 2,
 '0 00100000 00010000': 2,
 '0 10010000 11110000': 3,
 '1 10110010 10110000': 5,
 '0 11000000 10000000': 6,
 '0 11000000 11100000': 8,
 '0 00011000 00010000': 13,
 '0 11000000 11110000': 2,
 '1 10000001 10000000': 9,
 '1 00100010 00100000': 3,
 '0 10100000 10100000': 6037,
 '1 11100010 11100000': 1,
 '1 01010001 01010000': 6,
 '0 00100010 00100000': 5,
 '0 11010000 11100000': 3,
 '0 00000110 00000000': 4,
 '0 01100010 01100000': 3,
 '0 00000010 00000000': 3,
 '0 01110000 01010000': 12,
 '0 10101001 10100000': 4,
 '0 10000000 10100000': 9,
 '0 11110000 00110000': 7,
 '0 01000000 11000000': 3,
 '0 10110010 10110000': 3,
 '0 11010000 00010000': 1,
 '0 11010001 11010000': 6,
 '0 11110000 00010000': 1,
 '0 01100000 01000000': 8,
 '0 11011000 11010000': 8,
 '0 01110010 01110000': 2,
 '0 00000000 10000000': 7,
 '0 10110001 10110000': 4,
 '0 00010110 00010000': 1,
 '0 00101000 00100000': 18,
 '0 00000000 00010000': 3,
 '0 01010000 01100000': 6,
 '0 01101001 01100000': 2,
 '0 11100100 11100000': 4,
 '0 00110000 10110000': 5,
 '0 11110000 11100000': 3,
 '0 01001001 01000000': 2,
 '0 10011010 10010000': 4,
 '0 11010000 11000000': 7,
 '0 10000000 11100000': 3,
 '1 00000010 00000000': 2,
 '0 00010100 00010000': 4,
 '0 10110000 11110000': 12,
 '0 01011000 01010000': 10,
 '0 11010100 11010000': 3,
 '1 01000010 01000000': 7,
 '0 01100000 01110000': 5,
 '0 10010000 10100000': 3,
 '0 00100000 01000000': 2,
 '0 11010000 11010000': 6251,
 '0 11010111 11010000': 2,
 '1 10100001 10100000': 3,
 '0 01110110 01110000': 1,
 '1 11000010 11000000': 1,
 '0 11000000 11010000': 4,
 '0 01110000 00110000': 4,
 '0 01000000 01110000': 3,
 '0 11110110 11110000': 4,
 '0 11011001 11010000': 5,
 '0 11100000 11010000': 3,
 '0 11111000 11110000': 7,
 '0 01110000 01000000': 2,
 '1 10110001 10110000': 10,
 '0 00100000 00000000': 13,
 '0 11000000 00000000': 2,
 '1 00110010 00110000': 2,
 '0 11000001 11000000': 1,
 '0 11111001 11110000': 4,
 '0 10001001 10000000': 2,
 '0 00010000 00110000': 7,
 '0 10000000 10110000': 1,
 '0 10110000 10100000': 4,
 '0 11001010 11000000': 5,
 '0 10100000 10000000': 15,
 '0 10100100 10100000': 8,
 '0 00001000 00000000': 5,
 '0 01000000 10000000': 3,
 '0 00010000 00100000': 3,
 '0 00010000 01010000': 7,
 '0 01000000 01000000': 6238,
 '1 00000001 00000000': 2,
 '0 00100000 00100000': 6073,
 '0 01010000 00110000': 3,
 '0 00100001 00100000': 3,
 '0 10100000 11000000': 5,
 '0 10111001 10110000': 5,
 '1 11110001 11110000': 4,
 '0 00110000 00010000': 12,
 '1 01010010 01010000': 3,
 '0 01110000 01100000': 9,
 '0 00110000 01110000': 9,
 '0 00101001 00100000': 2,
 '0 01010000 10010000': 3,
 '1 11010001 11010000': 6,
 '0 01000001 01000000': 2,
 '1 01100001 01100000': 5,
 '0 11000000 01000000': 7,
 '0 11001001 11000000': 4,
 '0 10000110 10000000': 2,
 '0 11101000 11100000': 10,
 '0 11010000 10110000': 1,
 '0 01000000 01010000': 8,
 '0 10100010 10100000': 3,
 '0 11110000 01110000': 7,
 '1 01110001 01110000': 4,
 '0 11100110 11100000': 5,
 '0 11101001 11100000': 2,
 '0 10010000 10110000': 10,
 '0 00101010 00100000': 2,
 '0 10001000 10000000': 8,
 '0 01110000 00010000': 4,
 '0 00110110 00110000': 5,
 '1 11000001 11000000': 7,
 '0 00110000 11110000': 4,
 '0 01010100 01010000': 1,
 '0 01100111 01100000': 1,
 '0 01110100 01110000': 2,
 '0 10110000 10000000': 2,
 '0 11010000 10010000': 12,
 '0 00110010 00110000': 1,
 '0 10010010 10010000': 2,
 '0 01010000 00010000': 7,
 '0 11010000 01010000': 11,
 '0 00010010 00010000': 1,
 '0 00110000 01010000': 6,
 '0 10110100 10110000': 4,
 '0 11000110 11000000': 2,
 '0 01110000 01110000': 6253,
 '1 11010010 11010000': 1,
 '0 10101000 10100000': 5,
 '0 00010000 00000000': 8,
 '0 00100000 01100000': 6,
 '0 00100111 00100000': 1,
 '0 01110111 01110000': 3,
 '0 01010110 01010000': 4,
 '1 10000010 10000000': 2,
 '0 01100000 01100000': 6165,
 '0 11000111 11000000': 4,
 '0 11100000 11110000': 3,
 '0 01000110 01000000': 4,
 '0 10100001 10100000': 4,
 '0 10000001 10000000': 1}
# results = {x:y for x,y in results.items() if y > 3}
results = {'1 110110100 00110001 00110000':1}
tool.logical_error_rate(results, 0)

0.0

In [7]:
from qiskit import QuantumCircuit, execute, QuantumRegister, ClassicalRegister, Aer
from tqdm import tqdm
data = QuantumRegister(9, name='data')
mx = QuantumRegister(4, name='mx')
mz = QuantumRegister(4, name='mz')

rounds=2  # The actual number of rounds will always be more than 1, since the first round creates the quiescent state
measurements = [ClassicalRegister(8, name="c{}".format(i+1)) for i in range(rounds + 1)]
logical = QuantumRegister(1, name="logical")

base_circ = QuantumCircuit(data, mz, mx, *measurements, logical)

def stabilize(circ, i):
    # Top left
    circ.h(mx[0])
    circ.cx(mx[0], data[1])
    circ.cx(mx[0], data[0])
    circ.cx(data[1], mz[0])
    circ.cx(data[0], mz[0])
    circ.cx(data[4], mz[0])
    circ.cx(data[3], mz[0])
    circ.h(mx[0])
    
    # Top right
    circ.h(mx[1])
    circ.cx(mx[1], data[2])
    circ.cx(mx[1], data[1])
    circ.cx(mx[1], data[5])
    circ.cx(mx[1], data[4])
    circ.cx(data[2],mz[1])
    circ.cx(data[5],mz[1])
    circ.h(mx[1])
    
    # Bottom left
    circ.h(mx[2])
    circ.cx(data[3], mz[2])
    circ.cx(data[6], mz[2])
    circ.cx(mx[2], data[4])
    circ.cx(mx[2], data[3])
    circ.cx(mx[2], data[7])
    circ.cx(mx[2], data[6])
    circ.h(mx[2])

    # Bottom right
    circ.h(mx[3])
    circ.cx(mx[3], data[8])
    circ.cx(mx[3], data[7])
    circ.cx(data[5], mz[3])
    circ.cx(data[4], mz[3])
    circ.cx(data[8], mz[3])
    circ.cx(data[7], mz[3])
    circ.h(mx[3])
    circ.barrier()

    circ.measure(mz, measurements[i][0:4])
    circ.measure(mx, measurements[i][4:8])
    circ.reset(mz)
    circ.reset(mx)
    circ.barrier()

def get_stabilized_circ(base_circuit, rounds):
    circ = base_circuit.copy()
    for i in range(rounds + 1):
        stabilize(circ, i)
    return circ

In [8]:
circ = get_stabilized_circ(base_circ, rounds)
logical_zero = circ.copy()

logical_readout = ClassicalRegister(1, name="logicalR")
logical_zero.add_register(logical_readout)

# The Z-logical readout
logical_zero.reset(logical)
logical_zero.cx(data[0], logical)
logical_zero.cx(data[1], logical)
logical_zero.cx(data[2], logical)
logical_zero.measure(logical, logical_readout)

In [9]:
def get_noise_no_meas(p_err):

    error_gate1 = pauli_error([('X', p_err/2), ('Z', p_err/2), ('I', 1 - p_err)])
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_model = NoiseModel()
#     noise_model.add_all_qubit_quantum_error(error_gate1, "measure") # measurement error is applied to measurements
    noise_model.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"]) # single qubit gate error is applied to x gates
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"]) # two qubit gate error is applied to cx gates
        
    return noise_model

In [ ]:
logical = []
noise = [5e-5, 1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 4e-3, 5e-3, 6e-3, 7e-3, 8e-3, 9e-3, 1e-2, 2e-2]
for rate in tqdm(noise):
    results = execute(logical_zero, Aer.get_backend('qasm_simulator'), noise_model=get_noise_no_meas(rate), shots=1024*2).result().get_counts()
    logical.append(tool.logical_error_rate(results, 0))

 93%|█████████▎| 13/14 [03:14<00:27, 27.09s/it]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(noise, logical)
# plt.yscale("log")
# plt.xscale("log")